In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import widgets

sys.path.append('../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

In [ ]:
cache = True
auths = cb_utils.get_table( 'vw_ds_auth_mm'          , use_cache=cache)
claims = cb_utils.get_table('vw_ds_claims_mm'        , use_cache=cache)
visits = cb_utils.get_table('vw_ds_visit_features_mm', use_cache=cache)

In [ ]:
vcay =  cb_utils.get_table('vw_ds_all_visit_claims_auths_yr', use_cache=cache)
vcam =  cb_utils.get_table('vw_ds_all_visit_claims_auths_mm', use_cache=cache)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,5))
plt.suptitle('YEAR UTLZN')
axes[0].set_title('attd_pcs_utlzn')
vcay.query('attd_pcs_auth_hrs > 0 and attd_pcs_auth_utlzn < 110').attd_pcs_auth_utlzn.hist(bins=80, ax=axes[0])
axes[1].set_title('attd_utlzn')
vcay.query('attd_auth_hrs > 0     and attd_auth_utlzn     < 110').attd_auth_utlzn.hist(    bins=80, ax=axes[1])
axes[2].set_title('pcs_utlzn')
vcay.query('pcs_auth_hrs > 0      and pcs_auth_utlzn      < 110').pcs_auth_utlzn.hist(     bins=80, ax=axes[2])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,5))
plt.suptitle('Month UTLZN')
axes[0].set_title('attd_pcs_utlzn')
vcam.query('attd_pcs_auth_hrs > 0 and attd_pcs_auth_utlzn < 150').attd_pcs_auth_utlzn.hist(bins=80, ax=axes[0])
axes[1].set_title('attd_utlzn')
vcam.query('attd_auth_hrs > 0     and attd_auth_utlzn     < 150').attd_auth_utlzn.hist(    bins=80, ax=axes[1])
axes[2].set_title('pcs_utlzn')
vcam.query('pcs_auth_hrs > 0      and pcs_auth_utlzn      < 150').pcs_auth_utlzn.hist(     bins=80, ax=axes[2])

In [ ]:
# are pcs and atd hrs related?
plt.figure(figsize=(10, 10))
plt.scatter(visits.attd_hrs, visits.pcs_hrs, alpha=.1)

In [ ]:
claims.head()

In [ ]:
claims.describe()

In [ ]:
# PUll together
df = claims.merge(auths, how='left', on=['member_id', 'bom']).merge(visits, how='left', on=['member_id', 'bom'])
assert claims.shape[0] == df.shape[0]

In [ ]:
display(claims.shape)
display(auths.shape)
display(visits.shape)
display(df.shape)

In [ ]:
df.head()

In [ ]:
missed_hrs = df.groupby(['lob', 'ggroup', 'bom'], as_index=False).agg({'total_hrs_missed': 'median'})
sns.relplot(x="bom", y="total_hrs_missed", hue="ggroup", style="lob", data=missed_hrs);

In [ ]:
missed_hrs.bom.describe()

In [ ]:
pm = df.assign(pct_missed=df.total_hrs_missed / df.hrs)
pm.pct_missed = pm.pct_missed.fillna(0)
pm.loc[pm.lob == 1, ['lob']] = 'Medicaid'
pm.loc[pm.lob == 3, ['lob']] = 'DUAL'

# pm[['hrs', 'total_hrs_missed', 'pct_missed']]
sns.relplot(x="hrs", y="pct_missed", hue="ggroup", style="lob", data=pm, height=10, s=90);

In [ ]:
sns.relplot(x="hrs", y="pct_missed", size="ggroup", hue="lob", sizes=(15, 200), data=pm, height=10, alpha=0.3);

In [ ]:
# g = sns.FacetGrid(pm, col="lob", hue="ggroup", height=10)
g = sns.FacetGrid(pm, col="lob", row="ggroup", height=7)
g.map(plt.scatter, "hrs", "pct_missed", alpha=.3)
g.add_legend();

In [ ]:
df.head()

In [ ]:
ip = df.assign(pct_ip=df.ipat_$)

In [ ]:
za = cb_utils.get_table( 'vw_ds_all_z_yr' , use_cache=True)

In [ ]:
za

In [ ]:
# za[['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']])

In [ ]:
#sns.husl_palette(5, za['uzn_attd_pcs_q1_of_zs'])

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20,5))
plt.suptitle('attd_pcs_zscores')
axes[0].set_title('Q1')
za.uzn_attd_pcs_q1_of_zs.hist(bins=80, ax=axes[0])
axes[1].set_title('Q2')
za.uzn_attd_pcs_q2_of_zs.hist(bins=80, ax=axes[1])
axes[2].set_title('Q3')
za.uzn_attd_pcs_q3_of_zs.hist(bins=80, ax=axes[2])
axes[3].set_title('Q4')
za.uzn_attd_pcs_q4_of_zs.hist(bins=80, ax=axes[3])



In [ ]:
#Set marker properties
markercolor = za['uzn_attd_pcs_q4_of_zs'].round()
#markersize = za['uzn_attd_pcs_q4_of_zs']
#markersize = (za['uzn_attd_pcs_q4_of_zs'] * 5).round()

#Make Plotly figure
fig1 = go.Scatter3d(x=za['uzn_attd_pcs_q1_of_zs'],
                    y=za['uzn_attd_pcs_q2_of_zs'],
                    z=za['uzn_attd_pcs_q3_of_zs'],
                    marker=dict(color=markercolor,
                                size=5,
                                opacity=.25,
                                reversescale=True,
                                colorscale='Blues'
                                ),
                    line=dict (width=0.02),
                    mode='markers')

#Make Plot.ly Layout
mylayout = go.Layout(scene=dict(xaxis=dict( title="uzn_attd_pcs_q1_of_zs"),
                                yaxis=dict( title="uzn_attd_pcs_q2_of_zs"),
                                zaxis=dict(title="uzn_attd_pcs_q3_of_zs")),)

#Plot and save html
plotly.offline.plot({"data": [fig1], "layout": mylayout}, auto_open=True, filename=("4DPlot.html"))
#plotly.plot({"data": [fig1], "layout": mylayout});#, auto_open=True, filename=("4DPlot.html"))


In [ ]:
za = cb_utils.get_table( 'vw_ds_all_z_yr' , use_cache=True)
#x = (za[['attd_pcs_auth_hrs', 'uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']] * 2).round()/2
x = (za[['attd_pcs_auth_utlzn', 'attd_pcs_auth_hrs', 'attd_pcs_visit_hrs',  'uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']]).round()

#[x['uzn_attd_pcs_q1_of_zs'].unique().shape[0], x['uzn_attd_pcs_q2_of_zs'].unique().shape[0], x['uzn_attd_pcs_q3_of_zs'].unique().shape[0], x['uzn_attd_pcs_q4_of_zs'].unique().shape[0]]
#x['uzn_attd_pcs_q2_of_zs'].unique()


In [ ]:
# unique patterns
xx = x.groupby(['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']).count()

fig = px.parallel_coordinates(x[x.attd_pcs_auth_hrs > 200], color="attd_pcs_auth_hrs", labels={"attd_pcs_visit_hrs": "ZQ_1", "uzn_attd_pcs_q1_of_zs": "ZQ_2", "uzn_attd_pcs_q3_of_zs": "ZQ_3", "uzn_attd_pcs_q4_of_zs": "ZQ_4", },
                             color_continuous_scale=px.colors.diverging.Tealrose,
                              size = 'c'
                             color_continuous_midpoint=2)
fig.show()

In [ ]:
x[x[['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']].notnull().all(1)]

In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"
pio.renderers

In [ ]:
x['attd_pcs_auth_utlzn'] = x['attd_pcs_auth_utlzn'].round()

In [ ]:
#https://plotly.com/python/parallel-categories-diagram/

x = (za[['attd_pcs_auth_utlzn', 'attd_pcs_auth_hrs', 'attd_pcs_visit_hrs',  'uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']] * 2).round()/2
x['attd_pcs_auth_utlzn'] = x['attd_pcs_auth_utlzn'].round()
x['lob'] = za['lob']
x['grp'] = za['grp']

x2 = x[x[['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']].notnull().all(1)]
#x2 = x2.query('not (uzn_attd_pcs_q1_of_zs == 0 and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q2_of_zs and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q3_of_zs and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q4_of_zs)')

# Build parcats dimensions
categorical_dimensions = ['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']
dimensions = [dict(values=x2[label], label=label) for label in categorical_dimensions]

# Build colorscale
color = np.zeros(len(x2), dtype='uint8')
colorscale = [[0, 'light gray'], [1, 'firebrick']]

# Build figure as FigureWidget
fig = go.FigureWidget(
    data=[
    
          go.Scatter(x=x2.attd_pcs_auth_hrs, y=x2['attd_pcs_auth_utlzn'],
                     marker={"size": 5 * x2['grp'], "color":x2['attd_pcs_auth_utlzn']}, # , "cmid": 0},
                     mode='markers',
                     marker_colorscale=plotly.colors.sequential.Viridis,
                     # marker={'color': 'light blue'}, 
                     # selected={'marker': {'color': 'firebrick'}},
                     # unselected={'marker': {'color': 'attd_pcs_auth_utlzn', 'opacity': 0.4}}
          ), 
          
          go.Parcats(
              domain={'y': [0, 0.4]}, dimensions=dimensions,
              line={'colorscale': colorscale, 'cmin': 0,
                    'cmax': 3, 'color': color, 'shape': 'hspline'}
          )
        
    ])

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["type", "surface"],
                    label="3D Surface",
                    method="restyle"
                ),
                dict(
                    args=["type", "heatmap"],
                    label="Heatmap",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
        height=800, xaxis={'title': 'Auth Hrs'},
        yaxis={'title': 'Auth Utilization', 'domain': [0.6, 1]},
        dragmode='lasso', hovermode='closest')

# Update color callback
def update_color(trace, points, state):
    # Update scatter selection
    fig.data[0].selectedpoints = points.point_inds

    # Update parcats colors
    new_color = np.zeros(len(x2), dtype='uint8')
    new_color[points.point_inds] = 1
    fig.data[1].line.color = new_color
    fig.data[1].line

# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
# and parcats click
fig.data[1].on_click(update_color)

print(x2.shape)

fig

In [ ]:
#https://plotly.com/python/parallel-categories-diagram/


#cars_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/imports-85.csv')
#xx
x = (za[['attd_pcs_auth_utlzn', 'attd_pcs_auth_hrs', 'attd_pcs_visit_hrs',  'uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']] * 2).round()/2
x2 = x[x[['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']].notnull().all(1)]
x2 = x2.query('not (uzn_attd_pcs_q1_of_zs == 0 and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q2_of_zs and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q3_of_zs and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q4_of_zs)')
# Build parcats dimensions
categorical_dimensions = ['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']

dimensions = [dict(values=x2[label], label=label) for label in categorical_dimensions]

# Build colorscale
color = np.zeros(len(x2), dtype='uint8')
colorscale = [[0, 'gray'], [1, 'firebrick']]

# Build figure as FigureWidget
fig = go.FigureWidget(
    data=[
        go.Scatter(x=x2.attd_pcs_auth_hrs, y=x2['attd_pcs_auth_utlzn'],
    marker={'color': 'light blue'}, mode='markers', selected={'marker': {'color': 'firebrick'}},
    unselected={'marker': {'opacity': 0.4}}), go.Parcats(
        domain={'y': [0, 0.4]}, dimensions=dimensions,
        line={'colorscale': colorscale, 'cmin': 0,
              'cmax': 3, 'color': color, 'shape': 'hspline'})
    ])

fig.update_layout(
        height=800, xaxis={'title': 'Auth Hrs'},
        yaxis={'title': 'Auth Utilization', 'domain': [0.6, 1]},
        dragmode='lasso', hovermode='closest')

# Update color callback
def update_color(trace, points, state):
    # Update scatter selection
    fig.data[0].selectedpoints = points.point_inds

    # Update parcats colors
    new_color = np.zeros(len(cars_df), dtype='uint8')
    new_color[points.point_inds] = 1
    fig.data[1].line.color = new_color

# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
# and parcats click
fig.data[1].on_click(update_color)

print(x2.shape)

fig

In [ ]:
za['resp_auth_hrs'].describe()

In [ ]:
za[za.resp_visit_hrs > 10][['resp_visit_hrs','grp','lob']].groupby(['lob','grp'])['resp_visit_hrs'].describe()

In [ ]:
def build_fig(_df, _scatter_title, _scatter_x, _scatter_y, _ary_parcat_fields):


    # Build parcats dimensions
    categorical_dimensions = _parcat_fields
    dimensions = [dict(values= _df[label], label=label) for label in categorical_dimensions]

    # Build colorscale
    color = np.zeros(len(_df), dtype='uint8')
    colorscale = [[0, 'light gray'], [1, 'firebrick']]

    # Build figure as FigureWidget
    fig = go.FigureWidget(
        data=[

              go.Scatter(x=_df[_scatter_x], y=_df[_scatter_y],
                         marker={"size": 5 * _df['grp'], "color":_df[_scatter_y]}, # , "cmid": 0},
                         mode='markers',
                         marker_colorscale=plotly.colors.sequential.Viridis,
                         # marker={'color': 'light blue'}, 
                         # selected={'marker': {'color': 'firebrick'}},
                         # unselected={'marker': {'color': 'attd_pcs_auth_utlzn', 'opacity': 0.4}}
              ), 

              go.Parcats(
                  domain={'y': [0, 0.4]}, dimensions=dimensions,
                  line={'colorscale': colorscale, 'cmin': 0,
                        'cmax': 3, 'color': color, 'shape': 'hspline'}
              )

        ])

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=list([
                    dict(
                        args=["type", "surface"],
                        label="3D Surface",
                        method="restyle"
                    ),
                    dict(
                        args=["type", "heatmap"],
                        label="Heatmap",
                        method="restyle"
                    )
                ]),
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    fig.update_layout(
            height=800, xaxis={'title': _scatter_x },
            yaxis={'title': _scatter_title, 'domain': [0.6, 1]},
            dragmode='lasso', hovermode='closest')

    #print(self.df.shape)
    
    # Update color callback
    def update_color(trace, points, state):
        # Update scatter selection
        fig.data[0].selectedpoints = points.point_inds
    
        # Update parcats colors
        new_color = np.zeros(len(_df), dtype='uint8')
        new_color[points.point_inds] = 1
        print(1)
        fig.data[1].line.color = new_color
        fig.data[1].line

    # Register callback on scatter selection...
    fig.data[0].on_selection(update_color)
    # and parcats click
    fig.data[1].on_click(update_color)
        
    return fig

In [ ]:
#https://plotly.com/python/parallel-categories-diagram/
o_parcats_fields  = ['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']
o_scatter_field_x = 'attd_pcs_auth_hrs' 
o_scatter_field_y = 'attd_pcs_auth_utlzn'

x = (za[['attd_pcs_auth_utlzn', 'attd_pcs_auth_hrs', 'attd_pcs_visit_hrs',  'uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']] * 2).round()/2
x['attd_pcs_auth_utlzn'] = x['attd_pcs_auth_utlzn'].round()
x['lob'] = za['lob']
x['grp'] = za['grp']

x2 = x[x[['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']].notnull().all(1)]
#x2 = x2.query('not (uzn_attd_pcs_q1_of_zs == 0 and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q2_of_zs and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q3_of_zs and uzn_attd_pcs_q1_of_zs == uzn_attd_pcs_q4_of_zs)')

#x=x2.attd_pcs_auth_hrs, y=x2['attd_pcs_auth_utlzn'],
#fw = FancyWidget(x2)
#fw.df.head()
# _parcat_fields, _scatter_title, _scatter_x, _scatter_y
pcf = ["uzn_attd_pcs_q1_of_zs","uzn_attd_pcs_q2_of_zs","uzn_attd_pcs_q3_of_zs","uzn_attd_pcs_q4_of_zs"]

build_fig(x2, _scatter_title='Auth Utilization', _scatter_x='attd_pcs_auth_hrs', _scatter_y='attd_pcs_auth_utlzn', _parcat_fields=pcf )


#        x2.attd_pcs_auth_hrs, y=x2['attd_pcs_auth_utlzn'],

In [ ]:

# Build parcats dimensions
categorical_dimensions = parcats_fields
dimensions = [dict(values=self.df[label], label=label) for label in categorical_dimensions]

# Build colorscale
color = np.zeros(len(self.df), dtype='uint8')
colorscale = [[0, 'light gray'], [1, 'firebrick']]

# Build figure as FigureWidget
fig = go.FigureWidget(
    data=[

          go.Scatter(x=self.df.attd_pcs_auth_hrs, y=self.df[scatter_field_y],
                     marker={"size": 5 * df['grp'], "color":self.df[scatter_field_y]}, # , "cmid": 0},
                     mode='markers',
                     marker_colorscale=plotly.colors.sequential.Viridis,
                     # marker={'color': 'light blue'}, 
                     # selected={'marker': {'color': 'firebrick'}},
                     # unselected={'marker': {'color': 'attd_pcs_auth_utlzn', 'opacity': 0.4}}
          ), 

          go.Parcats(
              domain={'y': [0, 0.4]}, dimensions=dimensions,
              line={'colorscale': colorscale, 'cmin': 0,
                    'cmax': 3, 'color': color, 'shape': 'hspline'}
          )

    ])

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["type", "surface"],
                    label="3D Surface",
                    method="restyle"
                ),
                dict(
                    args=["type", "heatmap"],
                    label="Heatmap",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
        height=800, xaxis={'title': _scatter_x_axis },
        yaxis={'title': _scatter_title, 'domain': [0.6, 1]},
        dragmode='lasso', hovermode='closest')

# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
# and parcats click
fig.data[1].on_click(update_color)

print(self.df.shape)


# Update color callback
def update_color(trace, points, state):
    # Update scatter selection
    fig.data[0].selectedpoints = points.point_inds

    # Update parcats colors
    new_color = np.zeros(len(self.df), dtype='uint8')
    new_color[points.point_inds] = 1
    fig.data[1].line.color = new_color
    fig.data[1].line

fig


In [ ]:
za = cb_utils.get_table('vw_ds_all_z_yr' , use_cache=True)
x = (za[['attd_pcs_auth_utlzn',
         'attd_pcs_auth_hrs',
         'attd_pcs_visit_hrs',
         'uzn_attd_pcs_q1_of_zs',
         'uzn_attd_pcs_q2_of_zs',
         'uzn_attd_pcs_q3_of_zs',
         'uzn_attd_pcs_q4_of_zs'
        ]
       ] * 2).round()/2
x['attd_pcs_auth_utlzn'] = x['attd_pcs_auth_utlzn'].round()
x['lob'] = za['lob']
x['grp'] = za['grp']
x2 = x[x[['uzn_attd_pcs_q1_of_zs','uzn_attd_pcs_q2_of_zs','uzn_attd_pcs_q3_of_zs','uzn_attd_pcs_q4_of_zs']].notnull().all(1)]
# input #1: dataframe
df = x2.copy()
# input #2: scatter column inputs
x_label = 'attd_pcs_auth_hrs'
y_label = 'attd_pcs_auth_utlzn'
# input #3: boolean mask for buttons
mask = df['grp'] == 2
f = go.FigureWidget(plotly.subplots.make_subplots(rows=2))
# Do scatter stuff
f.add_scatter(x=df.loc[mask, x_label],
              y=df.loc[mask, y_label],
              marker={"size": 5 * df.loc[mask, 'grp'],
                      "color": df.loc[mask, y_label]
                     },
              mode='markers',
              marker_colorscale=plotly.colors.sequential.Viridis,
              name='Group 2'
              )
f.add_scatter(x=df.loc[~mask, x_label],
              y=df.loc[~mask, y_label],
              marker={"size": 5 * df.loc[~mask, 'grp'],
                      "color": df.loc[~mask, y_label]
                     },
              mode='markers',
              marker_colorscale=plotly.colors.sequential.Viridis,
              name='Group 3'
              )
# everything parcats ###################################################
# parcats inputs
categorical_dimensions = [f'uzn_attd_pcs_q{i}_of_zs' for i in range(1, 5)]
dimensions = [dict(values=df[label], label=label) for label in categorical_dimensions]
colorscale = [[0, 'light gray'], [1, 'firebrick']]
color = np.zeros(len(df), dtype='uint8')
f.add_parcats(
    domain={'y': [0, 0.4]},
    dimensions=dimensions,
    line={'colorscale': colorscale,
          'cmin': 0,
          'cmax': 3,
          'color': color,
          'shape': 'hspline'}
)
# everything parcats ###################################################
categorical_dimensions = [f'uzn_attd_pcs_q{i}_of_zs' for i in range(1, 5)]
dimensions = [dict(values=df.loc[mask, label], label=label) for label in categorical_dimensions]
colorscale = [[0, 'light gray'], [1, 'firebrick']]
color = np.zeros(len(df.loc[mask]), dtype='uint8')
f.add_parcats(
    domain={'y': [0, 0.4]},
    dimensions=dimensions,
    line={'colorscale': colorscale,
          'cmin': 0,
          'cmax': 3,
          'color': color,
          'shape': 'hspline'},
    visible=False
)
categorical_dimensions = [f'uzn_attd_pcs_q{i}_of_zs' for i in range(1, 5)]
dimensions = [dict(values=df.loc[~mask, label], label=label) for label in categorical_dimensions]
colorscale = [[0, 'light gray'], [1, 'firebrick']]
color = np.zeros(len(df.loc[~mask]), dtype='uint8')
f.add_parcats(
    domain={'y': [0, 0.4]},
    dimensions=dimensions,
    line={'colorscale': colorscale,
          'cmin': 0,
          'cmax': 3,
          'color': color,
          'shape': 'hspline'},
    visible=False
)
f.update_layout(
        height=800, xaxis={'title': 'attd_pcs_auth_hrs'},
        yaxis={'title': 'Auth Utilization', 'domain': [0.6, 1]},
        dragmode='lasso', hovermode='closest')
f.update_layout(
    updatemenus=[
        dict(buttons=list([
                dict(
                    args=[
                        {"type": ["scatter", "scatter", "parcats", "parcats", "parcats"],
                          "visible": [True, True, True, False, False]
                        }
                    ],
                    label="All",
                    method="restyle"
                ),
                dict(
                    args=[
                        {"type": ["scatter", "scatter", "parcats", "parcats", "parcats"],
                          "visible": [True, False, False, True, False],
                         "showlegend": [True, True, False, False, False]
                        }
                    ],
                    label="Group 2",
                    method="restyle"
                ),
                dict(
                    args=[
                        {"type": ["scatter", "scatter", "parcats", "parcats", "parcats"],
                         "visible": [False, True, False, False, True],
                         "showlegend": [True, True, False, False, False]
                        }
                    ],
                    label="Group 3",
                    method="restyle"
                )
            ]),
            type='buttons',
            direction="left",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)
# Update color callback
def update_color(trace, points, state):
    # Update scatter selection
    f.data[0].selectedpoints = points.point_inds
    f.data[1].selectedpoints = points.point_inds
    # Update parcats colors
    new_color = np.zeros(len(df), dtype='uint8')
    new_color[points.point_inds] = 1
    print(1)
    f.data[2].line.color = new_color
    f.data[2].line
    f.data[3].line.color = new_color
    f.data[3].line
    f.data[4].line.color = new_color
    f.data[4].line
    
# Register callback on scatter selection...
f.data[0].on_selection(update_color)
f.data[1].on_selection(update_color)
# and parcats click
f.data[2].on_click(update_color)
f.data[3].on_click(update_color)
f.data[4].on_click(update_color)
f